(1) II. 데이터 셋에서 주어진 특징들을 선택하여 CPU 성능 즉, “PRP” 변수를 예측하는 LinearRegression 기반 다중 선형 회귀 모델링을 프로그래밍 하시오.
- 데이터 탐색 및 시각화, 전처리, 훈련 데이터와 테스트 데이터 분리, 훈련 데이터 학습과정, 교차 검증, 분석 결과 시각화 등을 포함하여 최적화하는 전체 과정을 포함하시오.

In [77]:
import pandas as pd

# 데이터 로드
file_path = 'machine.data_update.csv'
data = pd.read_csv(file_path, header=None)

# 컬럼 이름 설정
data.columns = [
    'vendor', 'model', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'ERP'
]

# 데이터 샘플 확인
print(data.head())

# 데이터 요약 정보
print(data.info())

# 기술 통계 확인
print(data.describe())


print(data.dtypes)

data['MYCT'] = pd.to_numeric(data['MYCT'], errors='coerce')

# NaN 값 제거
data = data.dropna(subset=['MYCT'])

# NaN 값을 평균으로 대체
data['MYCT'] = data['MYCT'].fillna(data['MYCT'].mean())

# 범주형 변수 인코딩 (vendor와 model)
label_encoders = {}
for column in ['vendor', 'model']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# 특성과 타겟 변수 분리
X = data.drop(columns=['PRP'])
y = data['PRP']

# 데이터 분할 (훈련 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 초기화
model = LinearRegression()

# 교차 검증
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)

# 교차 검증 점수 출력
print("Cross-validation scores:", cv_scores)
print("Mean CV score:", cv_scores.mean())

# 모델 훈련
model.fit(X_train_scaled, y_train)

# 모델 훈련
model = LinearRegression()
model.fit(X_train, y_train)

# 교차 검증
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)

print(f'Cross-validated RMSE: {cv_rmse.mean()}')


       vendor      model  MYCT  MMIN   MMAX  CACH  CHMIN  CHMAX  PRP  ERP
0  VendorName  ModelName  MYCT  MMIN   MMAX  CACH  CHMIN  CHMAX  PRP  ERP
1     adviser      32/60   125   256   6000   256     16    128  198  199
2      amdahl     470v/7    29  8000  32000    32      8     32  269  253
3      amdahl    470v/7a    29  8000  32000    32      8     32  220  253
4      amdahl    470v/7b    29  8000  32000    32      8     32  172  253
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   vendor  210 non-null    object
 1   model   210 non-null    object
 2   MYCT    210 non-null    object
 3   MMIN    210 non-null    object
 4   MMAX    210 non-null    object
 5   CACH    210 non-null    object
 6   CHMIN   210 non-null    object
 7   CHMAX   210 non-null    object
 8   PRP     210 non-null    object
 9   ERP     210 non-null    object
dtypes: object(10)



(2) (1) 단계에서 얻은 다중 선형 회귀 모델을 사용하여 학습 데이터와 테스트 데이터 각각에 대해 이 모델의 성능을 평가하시오. 
- Mean squared Error, Mean absolute Error, R^2, 교차검증

In [78]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 학습 데이터 예측
y_train_pred = model.predict(X_train_scaled)

# 성능 지표 계산
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("Training set performance:")
print(f"Mean Squared Error: {train_mse:.2f}")
print(f"Mean Absolute Error: {train_mae:.2f}")
print(f"R-squared: {train_r2:.2f}")


Training set performance:
Mean Squared Error: 1524.00
Mean Absolute Error: 23.31
R-squared: 0.92


In [79]:
# 테스트 데이터 예측
y_test_pred = model.predict(X_test_scaled)

# 성능 지표 계산
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Test set performance:")
print(f"Mean Squared Error: {test_mse:.2f}")
print(f"Mean Absolute Error: {test_mae:.2f}")
print(f"R-squared: {test_r2:.2f}")


Test set performance:
Mean Squared Error: 2341.83
Mean Absolute Error: 31.46
R-squared: 0.95


In [80]:
# 교차 검증 성능 평가
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)

print(f'Cross-validated RMSE: {cv_rmse.mean():.2f}')


Cross-validated RMSE: 46.60


(3) (2) 단계에서 얻은 모델의 테스트 데이터에 대해 R^2가 90% 이상이 나오도록 모델을 개선하시오.

In [81]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.1, 1, 10, 100]}
ridge_model = Ridge()
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_scaled, y_train)

print(f"Test R-squared: {test_r2:.2f}")


Test R-squared: 0.95


(4) (1), (2), (3) 단계 과정을 포함한 전체 프로그래밍을 소스파일 “linear_cpu_data.ipynb”로 작성해서 사이버캠퍼스 “3차 프로그래밍과제”와 github에 각각 업로드 하시오.